# 1. RMSD between 2 molecules

In [2]:
from rmsd_calculator import GetBestRMSD
sdf_file1 = "/home/guest/AI_docking/docking_analysis_tool/3IK3_chainA_-_prepared_ligand.sdf"
sdf_file2 = "/home/guest/AI_docking/docking_analysis_tool/3IK3_chainA_-_prepared_ligand.sdf"


# here, rms is Fitted, rmsd is NOT Fit!!!
GetBestRMSD(sdf_file1,sdf_file2)

(0.0, 0.0)

In [45]:
from rdkit import Chem
# m = Chem.MolFromSmiles('OC1C2C1CC2')
m2 = Chem.AddHs(mol)
print("m Smiles:",Chem.MolToSmiles(mol))
print("m2 Smiles:",Chem.MolToSmiles(m2))
print("num ATOMs in m:",mol.GetNumAtoms())
print("num ATOMs in m2:",m2.GetNumAtoms())

m Smiles: O=C(Nc1ccc(OC(F)(F)Cl)cc1)c1cnc(N2CC[C@@H](O)C2)c(-c2ccn[nH]2)c1
m2 Smiles: [H]O[C@@]1([H])C([H])([H])N(c2nc([H])c(C(=O)N([H])c3c([H])c([H])c(OC(F)(F)Cl)c([H])c3[H])c([H])c2-c2c([H])c([H])nn2[H])C([H])([H])C1([H])[H]
num ATOMs in m: 31
num ATOMs in m2: 49


In [38]:
# 例子分子
# mol = Chem.MolFromSmiles('CCCCO')  # 乙醇

# 选取重原子（原子序数大于1）
heavy_atoms = [atom for atom in mol.GetAtoms()]

# 打印重原子的信息
for atom in heavy_atoms:
    print(f'原子编号: {atom.GetIdx()}, 元素: {atom.GetSymbol()}')

原子编号: 0, 元素: C
原子编号: 1, 元素: C
原子编号: 2, 元素: C
原子编号: 3, 元素: C
原子编号: 4, 元素: C
原子编号: 5, 元素: C
原子编号: 6, 元素: C
原子编号: 7, 元素: C
原子编号: 8, 元素: C
原子编号: 9, 元素: C
原子编号: 10, 元素: C
原子编号: 11, 元素: C
原子编号: 12, 元素: C
原子编号: 13, 元素: C
原子编号: 14, 元素: N
原子编号: 15, 元素: C
原子编号: 16, 元素: C
原子编号: 17, 元素: C
原子编号: 18, 元素: N
原子编号: 19, 元素: C
原子编号: 20, 元素: C
原子编号: 21, 元素: C
原子编号: 22, 元素: N
原子编号: 23, 元素: O
原子编号: 24, 元素: O
原子编号: 25, 元素: O
原子编号: 26, 元素: N
原子编号: 27, 元素: N
原子编号: 28, 元素: F
原子编号: 29, 元素: F
原子编号: 30, 元素: Cl


# 2. pocket extraction

In [8]:
from Bio import PDB
import numpy as np

def calculate_distance(atom1, atom2):
    """Calculate the Euclidean distance between two atoms."""
    diff_vector = atom1.coord - atom2.coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

def find_residues_around_ligand(pdb_file, ligand_name, distance_cutoff):
    """Find amino acid residues around a specific ligand within a given distance cutoff."""
    # Load the PDB structure
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('structure', pdb_file)
    water_and_ions = ['HOH', 'NA', 'CL', 'CA', 'MG', 'K']

    # Initialize lists to hold the results
    residues_within_cutoff = []

    # Iterate over all models in the structure (typically only one model)
    for model in structure:
        # Iterate over all chains in the model
        for chain in model:
            # print(chain.chain_id)
            # Find the ligand in the chain
            chain_id = chain.id
            ligand_residue = None
            for residue in chain:
                if residue.resname == ligand_name:
                    ligand_residue = residue
                    break

            if ligand_residue is None:
                continue

            # Iterate over all residues in the chain to find those within the distance cutoff
            for residue in chain:
                if residue == ligand_residue:
                    continue
                # 过滤非水分子和非离子的氨基酸残基
                if residue.get_resname() not in water_and_ions:
                # Calculate the minimum distance between any atom in the ligand and the residue
                    min_distance = min(calculate_distance(ligand_atom, residue_atom)
                                    for ligand_atom in ligand_residue
                                    for residue_atom in residue)

                    if min_distance <= distance_cutoff:
                        residue_id = residue.id[1]
                        residues_within_cutoff.append((f"{chain_id}-{residue.resname}-{residue_id}"))

    return residues_within_cutoff

def pocket_accuracy(res_group_cry, res_group_dock):
    unique_len=len(set(res_group_cry + res_group_dock))
    total_len=len(res_group_cry)+len(res_group_dock) 
    repeat_len=total_len-unique_len
    repeat_rate=repeat_len/len(res_group_cry)
    return repeat_rate

# 定义一个选择器类，只选择非水分子和非离子
class NonWaterIonSelector(PDB.Select):
    def accept_residue(self, residue):
        # 去除水分子 (HOH) 和常见离子 (NA, CL, CA, MG, K)
        water_and_ions = ['HOH', 'NA', 'CL', 'CA', 'MG', 'K']
        if residue.get_resname() in water_and_ions:
            return False
        else:
            return True
            

def main(pdb_file, ligand_name):
    # Find residues within 10Å and 5Å
    residues_within_10A = find_residues_around_ligand(pdb_file, ligand_name, 10.0)
    residues_within_5A = find_residues_around_ligand(pdb_file, ligand_name, 5.0)

    # Output the results
    print("Residues within 10Å:")
    for resname, resnum in residues_within_10A:
        print(f"{resname} {resnum}")

    print("\nResidues within 5Å:")
    for resname, resnum in residues_within_5A:
        print(f"{resname} {resnum}")


In [9]:
find_residues_around_ligand(pdb_file_cry, ligand_name, 5.0)

['A-LEU-248',
 'A-GLY-249',
 'A-TYR-253',
 'A-VAL-256',
 'A-ALA-269',
 'A-VAL-270',
 'A-LYS-271',
 'A-GLU-286',
 'A-MET-290',
 'A-VAL-299',
 'A-ILE-313',
 'A-ILE-314',
 'A-THR-315',
 'A-GLU-316',
 'A-PHE-317',
 'A-MET-318',
 'A-THR-319',
 'A-TYR-320',
 'A-GLY-321',
 'A-LEU-370',
 'A-ALA-380',
 'A-ASP-381']

In [4]:
pdb_file_cry = '/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/2GQG/2gqg_A.pdb'
res_group_dock = '/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/2GQG/2gqg_A.pdb'
ligand_name = '1N1'

res_group_cry = find_residues_around_ligand(pdb_file_cry, ligand_name, 5.0)
res_group_dock = find_residues_around_ligand(res_group_dock, ligand_name, 5.0)

pocket_accuracy(res_group_cry, res_group_dock)

1.0

# 3. Processing different docking systems

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBParser, Superimposer, PDBIO
import numpy as np

def read_sdf_file(file_path):
    """
    Reads a molecular file and returns a molecule object.

    Parameters:
        file_path (str): Path to the molecular file. Supported formats include .mol, .mol2, .sdf, .smiles.

    Returns:
        Chem.Mol: A molecule object. Raises an error if the file cannot be read or if the format is unsupported.
    """
    
    # Ensure the file exists
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    # Extract the file extension
    file_extension = os.path.splitext(file_path)[-1].lower()
    
    if file_extension == '.mol':
        mol = Chem.MolFromMolFile(file_path)
    elif file_extension == '.mol2':
        mol = Chem.MolFromMol2File(file_path)
    elif file_extension == '.sdf':
        # SDF files may contain multiple molecules; here we read only the first one
        supplier = Chem.SDMolSupplier(file_path)
        mol = next(iter(supplier), None)
    elif file_extension == '.smiles':
        with open(file_path, 'r') as f:
            smiles = f.read().strip()
        mol = Chem.MolFromSmiles(smiles)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")
    
    if mol is None:
        raise ValueError(f"Failed to read molecule from {file_path}")
    
    return mol

def read_pdb_file(pdb_path):
    """
    Reads a PDB file and returns a protein structure.

    Parameters:
        pdb_path (str): Path to the PDB file.

    Returns:
        Bio.PDB.Structure.Structure: A protein structure object.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_path)
    return structure

def combine_molecule_protein(mol, protein_structure):
    """
    Combines a small molecule with a protein structure.

    Parameters:
        mol (Chem.Mol): The small molecule.
        protein_structure (Bio.PDB.Structure.Structure): The protein structure.

    Returns:
        Bio.PDB.Structure.Structure: The combined structure of the molecule and protein.
    """
    # Convert the molecule to PDB format
    mol_pdb_block = Chem.MolToPDBBlock(mol)
    mol_structure = PDBParser(QUIET=True).get_structure('ligand', mol_pdb_block)
    
    # Combine the two structures
    combined_structure = protein_structure.copy()
    for chain in mol_structure.get_chains():
        combined_structure[0].add(chain)
    
    return combined_structure

def calculate_rmsd(mol1, mol2, mode="all_atoms"):
    """
    Calculates the RMSD between two molecules based on the specified mode.

    Parameters:
        mol1 (Chem.Mol): The first molecule.
        mol2 (Chem.Mol): The second molecule.
        mode (str): The mode for RMSD calculation. Can be "all_atoms", "heavy_atoms", or "alpha_c".

    Returns:
        float: The RMSD value.
    """
    if mode == "all_atoms":
        atoms1 = mol1.GetAtoms()
        atoms2 = mol2.GetAtoms()
    elif mode == "heavy_atoms":
        atoms1 = [atom for atom in mol1.GetAtoms() if atom.GetAtomicNum() > 1]
        atoms2 = [atom for atom in mol2.GetAtoms() if atom.GetAtomicNum() > 1]
    elif mode == "alpha_c":
        atoms1 = [atom for atom in mol1.GetAtoms() if atom.GetPDBResidueInfo().GetName().strip() == 'CA']
        atoms2 = [atom for atom in mol2.GetAtoms() if atom.GetPDBResidueInfo().GetName().strip() == 'CA']
    else:
        raise ValueError(f"Unsupported mode: {mode}")
    
    # Get coordinates
    coords1 = np.array([atom.GetOwningMol().GetConformer().GetAtomPosition(atom.GetIdx()) for atom in atoms1])
    coords2 = np.array([atom.GetOwningMol().GetConformer().GetAtomPosition(atom.GetIdx()) for atom in atoms2])

    # Calculate RMSD
    return np.sqrt(np.mean(np.sum((coords1 - coords2) ** 2, axis=1)))

def superimpose_proteins(protein1, protein2):
    """
    Superimposes two protein structures.

    Parameters:
        protein1 (Bio.PDB.Structure.Structure): The first protein structure.
        protein2 (Bio.PDB.Structure.Structure): The second protein structure.

    Returns:
        float: The RMSD value after superimposition.
    """
    sup = Superimposer()
    sup.set_atoms(list(protein1.get_atoms()), list(protein2.get_atoms()))
    sup.apply(protein2.get_atoms())
    return sup.rms

def process_input_files(input1, input2, mode="all_atoms"):
    """
    Main processing function for calculating RMSD based on input files.

    Parameters:
        input1 (str): Path to the first input file (.sdf or .pdb).
        input2 (str): Path to the second input file (.pdb).
        mode (str): The mode for RMSD calculation. Can be "all_atoms", "heavy_atoms", or "alpha_c".

    Returns:
        tuple: A tuple containing the RMSD values and their mean if input1 is .sdf, or a single RMSD value if input1 is .pdb.
    """
    if isinstance(input1, str) and input1.endswith('.sdf'):
        molecules = read_sdf_file(input1)
        protein_structure = read_pdb_file(input2)
        
        rmsd_values = []
        for mol in molecules:
            combined_structure = combine_molecule_protein(mol, protein_structure)
            rmsd = calculate_rmsd(mol, combined_structure, mode)
            rmsd_values.append(rmsd)
        
        rmsd_mean = np.mean(rmsd_values)
        return rmsd_values, rmsd_mean
    
    elif isinstance(input1, str) and input1.endswith('.pdb'):
        protein_structure1 = read_pdb_file(input1)
        protein_structure2 = read_pdb_file(input2)
        
        rms = superimpose_proteins(protein_structure1, protein_structure2)
        rmsd = calculate_rmsd(protein_structure1, protein_structure2, mode)
        
        return rms, rmsd

# Example usage
if __name__ == "__main__":
    input1 = "input1.sdf"  # or "input1.pdb"
    input2 = "input2.pdb"
    mode = "all_atoms"  # Can be "all_atoms", "heavy_atoms", or "alpha_c"
    
    result = process_input_files(input1, input2, mode)
    print(f"RMSD result: {result}")


In [2]:
equibind_file = "/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2"


In [13]:
### glob ###
import glob
for name in sorted(glob.glob('/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/*/*/*.pdb')):
    print(name)

### walk ###
import os
import fnmatch

pattern ="*.pdb"
for root,dirs,files in os.walk(equibind_file):
    # 匹配文件名
    print(files)
    # if fnmatch.fnmatch(file )


/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/2GQG/2gqg_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/2HYY/2hyy_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/3CS9/3cs9.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/3IK3/3ik3_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/3UE4/3ue4.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/5MO4_Nilotinib/5mo4_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/5MO4_asciminib/5mo4_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1_apo_docking/2GQG/2gqg_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1_apo_docking/2HYY/2gqg_A.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1_apo_docking/3CS9/2gqg_A.pdb
/home/guest/AI_docking/EquiBind-main/t

In [19]:

### walk ###
import os
import fnmatch

pattern ="*.pdb"
for files in os.listdir(equibind_file):
    # 匹配文件名
    # print(files)
    for file in files:
        if fnmatch.fnmatch(file, pattern):
            print(root,dirs,file)

In [20]:
def walkFile(file):
    for root, dirs, files in os.walk(file):

        # root 表示当前正在访问的文件夹路径
        # dirs 表示该文件夹下的子目录名list
        # files 表示该文件夹下的文件list

        # 遍历文件
        for f in files:
            print(os.path.join(root, f))

        # 遍历所有的文件夹
        for d in dirs:
            print(os.path.join(root, d))

'ABL1_apo_docking'

In [25]:

### walk ###
import os
import fnmatch

pattern ="*.pdb"
for root,dirs,files in os.walk(equibind_file):
    # 匹配文件名
    # print(files)
    for file in files:
        if fnmatch.fnmatch(file, pattern):
            print(root,dirs,file)

/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2 [] 4OGN_complex.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2_apo [] 5WTS_chainB_apo.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1 [] 6OQD_complex.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1_apo [] 6QB3_apo.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/BRD4_apo_docking/6UWU [] Protein_2oss.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/BRD4_apo_docking/4NUD [] Protein_2oss.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/TAF1/7L6X [] 7l6x.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/TAF1/7K27 [] 7k27.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/TAF1/7K0D [] 7k0d.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/TAF1/7K03 [] 7k

In [77]:

### walk ###
import os
import fnmatch


for root,dirs,files in os.walk(equibind_file):
    # 匹配文件名
    # print(files)
    for file in files:
        # 检查文件名是否以 "lig" 开头
        if not file.startswith('lig-'):
            # 输出文件的完整路径
            file_path = os.path.join(root, file)
            print(file_path)

/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2/4OGN_complex.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2/lig_equibind_corrected.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2/AM1900.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2_apo/5WTS_chainB_apo.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2_apo/lig_equibind_corrected.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2_apo/AM1900.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1/AMG-176.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1/lig_equibind_corrected.sdf
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1/6OQD_complex.pdb
/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MCL1

In [19]:
file = "sdsd.sa"

In [22]:
pattern = r"^(?!l)*.sdf"

In [23]:
fnmatch.fnmatch(file, pattern)

False

# 4.实际计算
## 4.1 Equibind RMSD

In [30]:
root

'/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1_apo_docking/3CS9'

In [80]:

### walk ###
import os
import pandas as pd
import fnmatch
from rmsd_calculator import GetBestRMSD

pattern_dock ="lig*.sdf"
sdf_dock = []
sdf_cry = []
rmsd_list = []

for root,dirs,files in os.walk(equibind_file):
    # 匹配文件名判断是否生产了对接过后的文件
    docking_file= 'lig_equibind_corrected.sdf'
    if docking_file in files:
        for file in files:
            if not file.startswith('lig') and file.endswith('sdf'):
                # 输出文件的完整路径
                file_path = os.path.join(root, file)
                sdf_cry.append(file_path)
            if fnmatch.fnmatch(file, pattern_dock):
                file_path = os.path.join(root, file)
                sdf_dock.append(file_path)

from rmsd_calculator import GetBestRMSD

for sdf_file1,sdf_file2 in zip(sdf_cry,sdf_dock):  
    sys_proname1 = sdf_file1.split("/")[-3]
    sys_pdbid1 = sdf_file1.split("/")[-2]
    sys_proname2 = sdf_file2.split("/")[-3]
    sys_pdbid2 = sdf_file2.split("/")[-2]
    if (sys_proname1 == sys_proname2) and (sys_pdbid1 == sys_pdbid2):    
        # here, rms is Fitted, rmsd is NOT Fit!!!
        rmsd_fit, rmsd_nofit = GetBestRMSD(sdf_file1,sdf_file2)
        rmsd_list.append([sys_proname1, sys_pdbid1, rmsd_nofit, rmsd_fit])

rmsd_pd = pd.DataFrame(rmsd_list,columns=["Protein","PDBid","RMSD","RMSD_fit"])

In [81]:
rmsd_pd

,Protein,PDBid,RMSD,RMSD_fit
0,others,MDM2,2.125514,1.182970
1,others,MDM2_apo,3.272007,2.286139
2,others,MCL1,9.609204,0.000050
3,others,MCL1_apo,11.173601,0.000047
4,BRD4_apo_docking,6UWU,74.732129,1.805987
5,BRD4_apo_docking,4NUD,62.882546,1.047369
6,TAF1,7L6X,14.641953,0.424929
7,TAF1,7K27,12.375467,0.207423
8,TAF1,7K0D,17.665120,1.060239
9,TAF1,7K03,238.253552,0.502316


In [72]:
sdf_cry[0]

'/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2/4OGN_complex.pdb'

In [68]:
sdf_dock[0]

'/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/others/MDM2/lig_equibind_corrected.sdf'

In [60]:
sys_pdbid2

'MCL1'

In [59]:
sys_pdbid1

'MDM2_apo'

In [58]:
sys_proname1 != sys_proname2

False

In [57]:
(sys_proname1!= sys_proname2) or (sys_pdbid1!=sys_pdbid2)

True

In [ ]:
from rmsd_calculator import GetBestRMSD

for sdf_file1,sdf_file2 in zip(sdf_cry,sdf_dock):        
    # here, rms is Fitted, rmsd is NOT Fit!!!
    GetBestRMSD(sdf_file1,sdf_file2)